In [1]:
%matplotlib inline
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [10]:
df_train=pd.read_csv('data/atec_anti_fraud_train.csv', parse_dates=['date'])

In [17]:
(df_train.date[0] - df_train.date[1]).days

47

In [3]:
df_train_clean=df_train.loc[df_train['label']!=-1]
del df_train

In [4]:
y=df_train_clean['label']
X=df_train_clean.drop(['id','label','date'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=69)

In [8]:
df_train_clean.columns

Index(['id', 'label', 'date', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',
       ...
       'f288', 'f289', 'f290', 'f291', 'f292', 'f293', 'f294', 'f295', 'f296',
       'f297'],
      dtype='object', length=300)

In [12]:
num_values=1000
category_feature=[]
for i in range(X.shape[1]):
    num = X['f'+str(i+1)].unique().shape[0]
    if num <=num_values:
        category_feature.append(i)

In [14]:
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [15]:
lgb_train=lgb.Dataset(x_train,y_train,free_raw_data=False)
lgb_valid=lgb.Dataset(x_test,y_test,reference=lgb_train,free_raw_data=False)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'train_metric': True,
    'num_leaves': 31,
    'early_stopping_round': 10,
    'learning_rate': 0.1,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'device': 'cpu',
    'verbose': 0
}
feature_name=['feasture_'+str(col+1) for col in range(x_train.shape[1])]

In [16]:
gbm=lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=[lgb_train, lgb_valid], valid_names=['train_set', 'valid_set'], feval=atec_metric, feature_name=feature_name, categorical_feature=category_feature)

/home/littleyy/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/littleyy/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 1

[1]	train_set's auc: 0.926018	train_set's atec_metric: 0.394958	valid_set's auc: 0.918703	valid_set's atec_metric: 0.37385
Training until validation scores don't improve for 10 rounds.
[2]	train_set's auc: 0.94948	train_set's atec_metric: 0.424817	valid_set's auc: 0.948873	valid_set's atec_metric: 0.406985
[3]	train_set's auc: 0.951046	train_set's atec_metric: 0.438144	valid_set's auc: 0.950603	valid_set's atec_metric: 0.410733
[4]	train_set's auc: 0.95269	train_set's atec_metric: 0.44641	valid_set's auc: 0.952268	valid_set's atec_metric: 0.428535
[5]	train_set's auc: 0.956853	train_set's atec_metric: 0.450073	valid_set's auc: 0.957902	valid_set's atec_metric: 0.423424
[6]	train_set's auc: 0.956918	train_set's atec_metric: 0.46456	valid_set's auc: 0.958156	valid_set's atec_metric: 0.439779
[7]	train_set's auc: 0.959028	train_set's atec_metric: 0.470853	valid_set's auc: 0.960312	valid_set's atec_metric: 0.442675
[8]	train_set's auc: 0.960319	train_set's atec_metric: 0.481558	valid_set's